In [1]:
import pandas as pd
import numpy as np
import backtest
import matplotlib.pyplot as plt 

from log_data import StockData

import os

from dotenv import load_dotenv
from log_data import StockData
from datetime import datetime

import sys
import backtest

# Baixando dados das ações
df = pd.read_parquet('cotacoes_acoes.parquet')
df_ibov = pd.read_parquet('ibov.parquet')
df_ibov.rename(columns={'indice': 'ticker', 'valor': 'preco_fechamento_ajustado'}, inplace=True)
df_ibov['ticker'] = df_ibov['ticker'].replace('IBOV', 'BOVA11')
df_smal =  pd.read_parquet('smll.parquet')
df_smal.rename(columns={'indice': 'ticker', 'valor': 'preco_fechamento_ajustado'}, inplace=True)
df_smal['ticker'] = df_smal['ticker'].replace('SMLL', 'SMAL11')
df_ibov['data'] = pd.to_datetime(df_ibov['data']); df_smal['data'] = pd.to_datetime(df_smal['data'])
df = pd.concat([df, df_ibov, df_smal], ignore_index=True)
df.fillna(0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650895 entries, 0 to 1650894
Data columns (total 12 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   data                       1650895 non-null  datetime64[ns]
 1   preco_abertura             1650895 non-null  float64       
 2   preco_fechamento           1650895 non-null  float64       
 3   preco_maximo               1650895 non-null  float64       
 4   preco_medio                1650895 non-null  float64       
 5   preco_minimo               1650895 non-null  float64       
 6   quantidade_negociada       1650895 non-null  float64       
 7   quantidade_negocios        1650895 non-null  float64       
 8   ticker                     1650895 non-null  object        
 9   volume_negociado           1650895 non-null  float64       
 10  fator_ajuste               1650895 non-null  float64       
 11  preco_fechamento_ajustado  1650895 no

In [2]:
# Dados ativos mais líquidos mes a mes
def valores_unicos_df(dataframe):
    valores_unicos = []
    for coluna in dataframe.columns[1:]:
        valores_unicos_coluna = list(dataframe[coluna].unique())
        valores_unicos.extend(valores_unicos_coluna)
    valores_unicos = list(set(valores_unicos))

    return valores_unicos

# Função para extrair o ano (primeiros 4 caracteres)
def extrair_ano(data):
    return data[:4]

# Função para extrair o valor após o hífen
def extrair_apos_hifen(data):
    partes = data.split('-')
    if len(partes) > 1:
        return partes[1]
    else:
        return None  

portfolio_liquidez = pd.read_csv('portfolio_quant.csv', delimiter=',', header=None)
tickers_strategy = valores_unicos_df(portfolio_liquidez)
portfolio_liquidez['Ano'] = portfolio_liquidez[0].apply(extrair_ano)
portfolio_liquidez['Mes'] = portfolio_liquidez[0].apply(extrair_apos_hifen)
portfolio_liquidez.tail(5)
portfolio_liquidez.shape
portfolio_liquidez = portfolio_liquidez.iloc[:, 0:5]

portfolio_liquidez.head(5)

,0,1,2,3,4
0,2000-1,PETR4,BBDC4,ELET6,CMIG4
1,2000-2,PETR4,CMIG4,BBDC4,ELET6
2,2000-3,PETR4,CMIG4,BBDC4,ELET6
3,2000-4,PETR4,BBDC4,CMIG4,ELET6
4,2000-5,PETR4,BBDC4,CMIG4,ELET6


In [3]:
## upload data
load_dotenv()
ambiente_local = os.getenv('AMBIENTE_MAC')
datapath_cdi = 'cdi_historico.xlsx'
cdi_data = backtest.cdidata_extract(ambiente_local+datapath_cdi)
print(type(cdi_data))
print(len(cdi_data))
print(cdi_data[0:20])

<class 'numpy.ndarray'>
166
[[Timestamp('2010-01-01 00:00:00') 0.0862 1 2010]
 [Timestamp('2010-02-01 00:00:00') 0.0862 2 2010]
 [Timestamp('2010-03-01 00:00:00') 0.0861 3 2010]
 [Timestamp('2010-04-01 00:00:00') 0.0938 4 2010]
 [Timestamp('2010-05-01 00:00:00') 0.0937 5 2010]
 [Timestamp('2010-06-01 00:00:00') 0.1012 6 2010]
 [Timestamp('2010-07-01 00:00:00') 0.1063 7 2010]
 [Timestamp('2010-08-01 00:00:00') 0.1063 8 2010]
 [Timestamp('2010-09-01 00:00:00') 0.1061 9 2010]
 [Timestamp('2010-10-01 00:00:00') 0.1064 10 2010]
 [Timestamp('2010-11-01 00:00:00') 0.1064 11 2010]
 [Timestamp('2010-12-01 00:00:00') 0.1065 12 2010]
 [Timestamp('2011-01-01 00:00:00') 0.1114 1 2011]
 [Timestamp('2011-02-01 00:00:00') 0.1162 2 2011]
 [Timestamp('2011-03-01 00:00:00') 0.1172 3 2011]
 [Timestamp('2011-04-01 00:00:00') 0.119 4 2011]
 [Timestamp('2011-05-01 00:00:00') 0.1205 5 2011]
 [Timestamp('2011-06-01 00:00:00') 0.1221 6 2011]
 [Timestamp('2011-07-01 00:00:00') 0.124 7 2011]
 [Timestamp('2011-08-

In [4]:
# Objeto dados das ações
stock_data = StockData(df, tickers_selecionados=tickers_strategy)
print(len(tickers_strategy))
print(type(tickers_strategy))
print(stock_data.price_matrix.shape)
valores_nao_comuns = [valor for valor in tickers_strategy if valor not in stock_data.tickers]
weekdays = [StockData.date_to_weekday(date) for date in stock_data.dates]
print(weekdays[:5])
print(valores_nao_comuns)

86
<class 'list'>
(3426, 85)
['segunda', 'terça', 'quarta', 'quinta', 'sexta']
['XPBR31']


In [5]:
# Verificação Ativos Início e Fim
assets_trading = 'Ativos backtest: '
for ticker in stock_data.tickers:
    assets_trading += ticker+' , '

print(assets_trading, '\n')
print('Inicio Backtest: ', stock_data.dates[0])
print('Fim do Backtest: ', stock_data.dates[-1])
print('Return Benchmark _ Strategy Long Bova11: ', round( stock_data.price_matrix[-1,stock_data.tickers.index('BOVA11')]/ stock_data.price_matrix[0,stock_data.tickers.index('BOVA11')] - 1, 2))
print('Return Benchmark _ Strategy Long Bova11: ', round( stock_data.price_matrix[-1,stock_data.tickers.index('SMAL11')]/ stock_data.price_matrix[0,stock_data.tickers.index('SMAL11')] - 1, 2),'\n')

Ativos backtest: ITSA4 , VALE3 , EMBR3 , EQTL3 , BRFS3 , PETR3 , HYPE3 , PETR4 , ETER3 , LIGT3 , RAPT4 , CCRO3 , ELET6 , ELET3 , BRKM5 , TUPY3 , GGBR4 , MRFG3 , GFSA3 , USIM5 , BRAP4 , WEGE3 , GRND3 , GOLL4 , GOAU4 , LREN3 , MRVE3 , FESA4 , MULT3 , MYPK3 , FLRY3 , POSI3 , BBDC3 , SANB11 , ROMI3 , ITUB4 , SHUL4 , CYRE3 , CSNA3 , CPFE3 , CPLE6 , ALPA4 , VULC3 , JBSS3 , BBAS3 , DASA3 , BBDC4 , UNIP6 , TOTS3 , SBSP3 , CMIG4 , KEPL3 , RENT3 , CIEL3 , CSAN3 , RANI3 , UGPA3 , BRPR3 , ECOR3 , MGLU3 , QUAL3 , RADL3 , BBSE3 , ABEV3 , KLBN11 , CVCB3 , PRIO3 , BPAC11 , RAIL3 , AZUL4 , CRFB3 , IRBR3 , SUZB3 , B3SA3 , HAPV3 , COGN3 , NTCO3 , CASH3 , RRRP3 , RDOR3 , ASAI3 , AMER3 , VBBR3 , BOVA11 , SMAL11 ,  

Inicio Backtest:  2010-01-04 00:00:00
Fim do Backtest:  2023-10-30 00:00:00
Return Benchmark _ Strategy Long Bova11:  0.61
Return Benchmark _ Strategy Long Bova11:  0.64 



In [6]:
# backtest
contagem = np.zeros(stock_data.price_matrix.shape[1])
capital_alocacao = np.zeros((stock_data.price_matrix.shape[0], stock_data.price_matrix.shape[1] + 1))
resultado_alocacao = np.zeros((stock_data.price_matrix.shape[0], stock_data.price_matrix.shape[1] + 1))
capital_alocacao[:, -1] = 1

# Alocação portfolio
retorno_treshold = 0.01
periodo_position = 100 # input("Nr de dias posicionado: ")
cash = 1

for i, day in enumerate(stock_data.dates):

    # identificar mes e ano
    ano = day.year; mes = day.month
    
    # pegar a linha do joao com o portfolio desejado
    stocks2trade = portfolio_liquidez[(portfolio_liquidez['Ano'] == str(ano)) & (portfolio_liquidez['Mes'] == str(mes))]    
    if not stocks2trade.empty:
        stocks2trade = stocks2trade.iloc[0].tolist()
    else:
        stocks2trade = []
        print('Falta mes e ano na portfolio de liquidez')
        sys.exit()        


    # verificar entradas e saídas e compor portfolio

    for i2, ticker in enumerate(stocks2trade[1:-2]): # passar pelas 20 ações mais líquidas e identificar queda: 

        # posicao ticker price matrix
        if ticker != 'XPBR31':
            if stock_data.price_matrix[i-5, stock_data.tickers.index(ticker)] != 0: # se o papel existir nesse mes

                # tratamento posição já existente:
                capital_alocacao[i, stock_data.tickers.index(ticker)] = capital_alocacao[i-1, stock_data.tickers.index(ticker)]
                if capital_alocacao[i-1, stock_data.tickers.index(ticker)] != 0:
                    contagem[stock_data.tickers.index(ticker)] += 1
                if contagem[stock_data.tickers.index(ticker)] > int(periodo_position):
                    capital_alocacao[i-1, stock_data.tickers.index(ticker)] = 0
                    contagem[stock_data.tickers.index(ticker)] = 0

                # verificar oportunidade de nova operação:
                if weekdays[i]=='sexta': # se for sexta

                    weekly_return = backtest.calculate_return(stock_data.price_matrix[i-5, stock_data.tickers.index(ticker)], stock_data.price_matrix[i-1, stock_data.tickers.index(ticker)])
                    if weekly_return < retorno_treshold * -1:
                        capital_alocacao[i, stock_data.tickers.index(ticker)], cash = backtest.simple_update_capital(cash, 'buy')
                        contagem[stock_data.tickers.index(ticker)] = 0
                    else:
                        continue

    if cash < -0.001:
        print('SYSTEM ERROR - cash out of reality')
        print(cash)
        sys.exit()
    else:
        cash = 1 - np.sum(capital_alocacao[i, :-1])
        capital_alocacao[i, -1] = cash




#         if stock_data.price_matrix[i-5, i2] != 0:
#             # tratamento posição já existente
#             capital_alocacao[i, i2] = capital_alocacao[i-1, i2]   
#             if capital_alocacao[i-1, i2] != 0: # se posicionado, conta-se dias posicionados
#                 contagem[i2] += 1
#             if contagem[i2] > int(periodo_position): # se tempo de posição se excede vende
#                 capital_alocacao[i, i2] = 0
#                 contagem[i2] = 0
#             # verificação para abertura de nova posição
#             if weekdays[i] == 'sexta': # pode ser segunda tbm
#                 weekly_return = backtest.calculate_return(stock_data.price_matrix[i-5, i2], stock_data.price_matrix[i-1, i2])
#                 if weekly_return < retorno_treshold*-1:
#                     capital_alocacao[i, i2], cash = backtest.simple_update_capital(cash, "buy")
#                     contagem[i2] = 0
#             else:
#                 continue

#     if cash < -0.05:
#         print('SYSTEM ERROR - cash out of reality')
#         print(cash)
#         sys.exit()
#     else:
#         cash = 1 - np.sum(capital_alocacao[i, :-1])
#         capital_alocacao[i, -1] = cash

# print('\n',capital_alocacao[30:60, 0:6])     

KeyError: 'Ano'

In [ ]:
plt.plot(capital_alocacao[:,-1])

In [ ]:
print('\n',capital_alocacao[30:60, 0:10])

In [ ]:
# Building Equity Curve
equity =[100]

for i, day in enumerate(stock_data.dates[1:], start=1):

    for i2, ticker in enumerate(stock_data.tickers):

        if stock_data.price_matrix[i-1, i2] != 0: 

            position_ticker = capital_alocacao[i, i2]

            day_return = backtest.calculate_return(stock_data.price_matrix[i-1, stock_data.tickers.index(ticker)], stock_data.price_matrix[i, stock_data.tickers.index(ticker)])

            resultado_alocacao[i, i2] = position_ticker*day_return

    equity.append((np.sum(resultado_alocacao[i, :])) * equity[i-1] + equity[i-1])

In [ ]:
plt.plot(equity)

In [ ]:
# Building Equity Curve - Benchmark
equity_benchmark =[100]

for i, day in enumerate(stock_data.dates[1:], start=1):

    day_return = backtest.calculate_return(stock_data.price_matrix[i-1, 6], stock_data.price_matrix[i, 6])
    
    equity_benchmark.append(day_return * equity_benchmark[i-1] + equity_benchmark[i-1])